In [9]:
function generate_sequences()
end

generate_sequences (generic function with 1 method)

In [10]:
# test generate_sequences here

In [11]:
function MSA_to_TSP()
end

MSA_to_TSP (generic function with 1 method)

In [12]:
# test MSA_to_TSP here

In [13]:
function get_alignment_score()
end

get_alignment_score (generic function with 1 method)

In [14]:
# test get_alignment_score here

In [15]:
function TSP_to_MSA()
end

TSP_to_MSA (generic function with 1 method)

In [16]:
# test TSP_to_MSA here